<a href="https://colab.research.google.com/github/rypotter/depo/blob/master/Copy_of_Chat_with_MultiplePDFs_Mistral_7B_Instruct1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mistral-7B-Instruct Multiple-PDF Chatbot with Langchain & Streamlit |FREE COLAB|All OPEN SOURCE**
https://www.youtube.com/watch?v=tqpXvPzteT4 DataInsightEdge

https://mistral.ai/news/announcing-mistral-7b/

# **Copy to Drive**
Áttérünk a Drive-ra, bár mehetne a Github-ra is

# **Model letöltése:**

https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/tree/main
**mistral-7b-v0.1.Q4_K_M.gguf 4.37GB** ebben a sorban a lefelé mutató nyil.

Model feltöltése a Drive-ra (Github nem enged feltölteni 25MB-nél nagyobbat)
**NEM LEHET lokálisan használni!**

**walmart-inc-2023-annual-report.pdf** feltöltése a Drive-ra a MyDrive/Colab Notebooks/data/ könyvtárba


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Colab Notebooks/data/")
data = loader.load()

In [ ]:
print(data)

[Document(page_content='Annual Repor/t.ss04\n', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/walmart-inc-2023-annual-report.pdf', 'page': 0}), Document(page_content='Read the full repor/t.ss04 and Doug’s letter  here:Doug McMillon \nPresident and Chief Executive Officer  \nWalmar/t.ss04 Inc.“We started this new year with momentum across the business through \nstrength in our omnichannel model and by living our purpose every day. \nOur customers and members expect to shop with us when and how they choose, and that’s what we do for them. We’re thankful to our associates that create a seamless experience that’s uniquely Walmart.”A message from our CEO\n', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/walmart-inc-2023-annual-report.pdf', 'page': 1}), Document(page_content="UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________\nFORM \n10-K\n___________________________________________ \n☒A

# New Section

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

100

In [ ]:
#get the third chunk
text_chunks[2]

Document(page_content="UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________\nFORM \n10-K\n___________________________________________ \n☒Annual report pursuant to section 13 or 15(d) of the Securities Exchange Act of 1934\nFor the fiscal year ended January 31, 2023, or\n☐Transition report pursuant to section 13 or 15(d) of the Securities Exchange Act of 1934\nCommission file number 001-06991.\n ___________________________________________ \n \nWALMART INC.\n(E\nxact name of registrant as specified in its charter)\n___________________________________________ \nDE 71-0415188\n(State or other jurisdiction of\nincorporation or organization)(IRS Employer Identification No.)\n702 S.W. 8th Street\n72716 Bentonville, AR\n(Address of principal executive offices) (Zip Code)\nRegistrant's telephone number, including area code: (479) 273-4000 \nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class Trading S

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/Colab Notebooks/model/mistral-7b-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
# query = "What is linear regression model"
query = "Hello"

In [ ]:
qa.run(query)

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit


Answer:  It is a statistical method used to analyze the linear relationship between two or more variables. In this case, it is used to estimate the coefficients of a linear regression model.


Llama.generate: prefix-match hit


Answer:  sandwich provides various functions for estimating the covariance matrix. The most commonly used ones are vcovHAC and vcovHC. These functions can take different weighting schemes, including kernel-based HAC estimation with automatic bandwidth selection based on weightsAndrews andbwAndrews. Other functions like weightsLumley provide different weighting schemes, such as truncated and smoothed weights, which are useful in certain applications. In econometric analyses, these functions are used to compute partial t-tests for assessing the significance of a parameter. The choice of estimator depends on the presence of heteroscedasticity and autocorrelation in the errors.
